## Prática: Optical Caracter Recognition

Nesta prática será feita uma investigação sobre a base de dados, bem como identificação de ruídos. Na sequência será feita seleção de atributos e transformações sobre eles.
[Maiores detalhes sobre a prática](https://docs.google.com/document/d/19hujByNpyvDqJYW2QCeU-ecJK4uSLTslBXaouM9N05c/edit?usp=sharing)

## Antes de começar...

**Dependencias:** instalar Weka.

## Parte 1 - Explorando os dados

Sua primeira tarefa é aprender sobre a coleção e verificar ruídos nas mesmas

**Atividade 1**: Observe os histogramas (distribuições de classe por atributos). No pandas, existe o método describe e, no Weka, os histogramas estão na aba de preprocessamento e os gráficos  de espalhamento na aba “Visualize” (não esqueça de aplicar o jitter). Olhe também as instâncias usando o botão edit e tente visualiza-la por meio de um heatmap1. Como o dígito está sendo representado nesta imagem (ou seja, qual cor é o fundo e qual a é a cor do dígito)? Você percebe algo de incomum em alguma instância? Existe alguma instância com atributos que podem representar ruídos? Descreva qualquer características que lhe chame a atenção (dica: pense que valores você esperaria nos pixels de diferentes áreas da imagem).  


**Atividade 2**: No Weka, avalie esta coleção “mnist_avaliacao.csv” usando validação cruzada com 5 partições. Utilize os métodos J48 e NaiveBayes. Qual a precisão geral do método? Apresente a matriz de confusão. Quais conclusões você tira a partir da matriz de confusão de cada método? Quais classes são mais fácieis de prever? E a mais difícieis? Quais são os erros mais comuns em cada metodo?


## Parte 2 - Seleção e engenharia de atributos

Agora vamos melhorar o modelo explorando os atributos.

**Atividade 1**: No Weka, em Select Atributes, calcule o infogain (Ganho de Informação) de cada atributo. Vá a aba de visualização e observe os gráficos de dispersão entre os atributos mais bem e pior ranqueados contra a classe. Também analise os  histogramas na aba preprocessamento. Que diferença você nota? Qual região que possui os pixels mais importantes? (Dica: Tente também plotar os valores de  infogain de uma forma que fique fácil de identificar a região que cada pixel representa use um heatmap2).


**Atividade 2**: Analise os atributos individuais na aba preprocessamento além do infogain de cada atributo. Identifique os atributos que não são úteis ao processo de classificação. Remova estes atributos descreva e justifique as remoções. Armazene o dataset em “mnist_sample_selecaoFeature.csv”. (dica: Analisando o infogain, você pode selecionar apenas os mais importantes alterando os parametros do “Ranker”: Clique em ranker, mude o threshold para eliminar os valores de infogain que achar pertinente. Logo após, copie a lista de “selected attributes”. Vá na aba de preprocessamento, utilize o filtro não supervisionado de atributos “Remove” com esta lista não esqueça de inverter a seleção e cuidado para não remover a classe.).Armazene o dataset em “mnist_sample_selecaoFeature.csv”. 


**Atividade 3**: Re-avalie a base usando NB e  J48 no Weka. Compare o resultado com o que você obteve em I(d). Qual foi o benefício de realizar esta remoções?


## Parte 3 - Transformação de atributos

Usando o dataset “mnist_sample_selecaoFeature.arff”. Mude os atributos de numéricos para booleanos. Então re-avalie utilizando  NB e J48. (Dica: Utilize o filtro de atributos NumericToBinary).

**Atividade 1**: Em comparação com o resultado em 2(c), houve ganho no desempenho? Analise e reporte a diferença do desempenho por classe e geral. Com base no que se aprendeu sobre árvore de decisão e a forma que ela é criada para atributos numéricos, justifique o impacto utilizando J48. Armazene o dataset em “mnist_sample_binario.arff”.


## Parte 4 - Resultado no teste
Faça todo o processamento que você fez no treino/validação no teste ou seja: elimine os atributos que você julgou necessário em (2) e faça a mesma transformação de (3). Apresente o resultado obtido.

[Utilize CNN no Keras](https://keras.io/examples/vision/mnist_convnet/) na base de dados testada e compare os resultados de cada fase avaliada (exercícios 1, 2 e 3).